# Simulación de una avenida con paso peatonal
En esta simulación se representa una avenida con dos carriles y un paso peatonal. El tráfico se controla con dos semáforos que detienen a los autos en el paso peatonal.

## Librerías para la simulación

In [18]:
import agentpy as ap
import numpy as np
import matplotlib.pyplot as plt
import math
import IPython
import json
from pprint import pprint
from collections import defaultdict

## Definición del agente semáforo

In [10]:
class Semaphore(ap.Agent):
    """ 
        Esta clase define a un semáforo.
    """

    def setup(self):
        """ Este método se utiliza para inicializar al semáforo. """
        
        self.step_time = 0.3         # Tiempo que dura cada paso de la simulación

        self.direction = [0, 1]      # Dirección a la que apunta el semáforo

        self.state = 0               # Estado del semáforo 0 = verde, 1 = amarillo, 2 = rojo
        self.state_time = 0          # Tiempo que ha durado el semáforo en el estado actual

        self.green_duration = 50     # Tiempo que dura el semáforo en verde
        self.yellow_duration = 5     # Tiempo que dura el semáforo en amarillo
        self.red_duration = 45       # Tiempo que dura el semáforo en rojo        

    def update(self):
        """ Este método actualiza el estado del semáforo. """
        self.state_time += self.step_time

        if self.state == 0:
            # Caso en el que el semáforo está en verde
            if self.state_time >= self.green_duration:
                self.state = 1
                self.state_time = 0
        elif self.state == 1:
            # Caso en el que el semáforo está en amarillo
            if self.state_time >= self.yellow_duration:
                self.state = 2
                self.state_time = 0
        elif self.state == 2:
            # Caso en el que el semáforo está en rojo
            if self.state_time >= self.red_duration:
                self.state = 0
                self.state_time = 0

    def set_green(self):
        """ Este método forza el semáforo a estar en verde. """        
        self.state = 0
        self.state_time = 0

    def set_yellow(self):
        """ Este método forza el semáforo a estar en amarillo. """        
        self.state = 1
        self.state_time = 0

    def set_red(self):
        """ Este método forza el semáforo a estar en rojo. """        
        self.state = 2
        self.state_time = 0


## Definición del agente auto

In [130]:
class Car(ap.Agent):
    """ 
        Esta clase define a un auto.
    """

    def setup(self):
        """ Este método se utiliza para inicializar un robot limpiador. """
        self.step_time = 0.1         # Tiempo que dura cada paso de la simulación
        self.current_frame = 0
        self.direction = [1, 0]      # Dirección a la que viaja el auto
        self.speed = 0.0             # Velocidad en metros por segundo
        self.max_speed = 20          # Máxima velocidad en metros por segundo
        self.state = 1               # Car state: 1 = ok, 0 = dead
        
        
        
    def update_position(self):
        """ Este método se utiliza para inicializar la posición del auto. """

        # Verifica si el auto no ha chocado
        if self.state == 0:
            return

         # Actualiza la posición según la velocidad actual
        self.model.avenue.move_by(self, [self.speed*self.direction[0], self.speed*self.direction[1]])

    def update_speed(self):
        
        """ Este método se utiliza para inicializar la velocidad del auto. """

        # Verifica si el auto no ha chocado
        if self.state == 0:
            return
        
        # Obten la distancia más pequeña a uno de los autos que vaya en la misma dirección        
        p = self.model.avenue.positions[self]

        min_car_distance = 1000000
        for car in self.model.cars:
            if car != self:
                # Verifica si el carro va en la misma dirección
                dot_p1 = self.direction[0]*car.direction[0] + self.direction[1]*car.direction[1]                
                
                # Verifica si el carro está atrás o adelante
                p2 = self.model.avenue.positions[car]
                dot_p2 = (p2[0]-p[0])*self.direction[0] + (p2[1]-p[1])*self.direction[1]

                if dot_p1 > 0 and dot_p2 > 0:                     
                    d = math.sqrt((p[0]-p2[0])**2 + (p[1]-p2[1])**2)                    
                    
                    if min_car_distance > d:
                        min_car_distance = d
       
        # Obten la distancia al próximo semáforo
        min_semaphore_distance = 1000000
        semaphore_state = 0
        for semaphore in self.model.semaphores:

            # Verifica si el semáforo apunta hacia el vehículo
            dot_p1 = semaphore.direction[0]*self.direction[0] + semaphore.direction[1]*self.direction[1]
            
            # Verifica si el semáforo está adelante o atrás del vehículo
            p2 = self.model.avenue.positions[semaphore]
            dot_p2 = (p2[0]-p[0])*self.direction[0] + (p2[1]-p[1])*self.direction[1]

            if dot_p1 < 0 and dot_p2 > 0:                            
                d = math.sqrt((p[0]-p2[0])**2 + (p[1]-p2[1])**2)  
                
                if min_semaphore_distance > d:
                    min_semaphore_distance = d
                    semaphore_state = semaphore.state
        
        # Actualiza la velocidad del auto
        if min_car_distance < 2:
            self.speed = 0
            self.state = 1

        elif min_car_distance < 20:
              self.speed = np.maximum(self.speed - 250*self.step_time, 0)

        elif min_car_distance < 50:
              self.speed = np.maximum(self.speed - 120*self.step_time, 0)
                
        elif min_semaphore_distance < 40 and semaphore_state == 1:
            self.speed = np.minimum(self.speed + 5*self.step_time, self.max_speed)

        elif min_semaphore_distance < 50 and semaphore_state == 1:
            self.speed = np.maximum(self.speed - 25*self.step_time, 0)
            
        elif min_semaphore_distance < 120 and semaphore_state == 2:
            self.speed = np.maximum(self.speed - 90*self.step_time, 0)

        else:
            self.speed = np.minimum(self.speed + 5*self.step_time, self.max_speed)
                    


## Definición del modelo de la avenida

In [131]:
class AvenueModel(ap.Model):
    """ Esta clase define un modelo para una avenida simple con semáforo peatonal. """

    def setup(self):
        """ Este método se utiliza para inicializar la avenida con varios autos y semáforos. """
        
        # Inicializa los agentes los autos y los semáforos        
        self.cars = ap.AgentList(self, self.p.cars, Car)
        self.cars.step_time =  self.p.step_time
        self.frame = 0
        self.count = 0
        c_north = int(self.p.cars/2)
        c_south = self.p.cars - c_north

        for k in range(c_north):
            self.cars[k].direction = [0,1]

        for k in range(c_south):
            self.cars[k+c_north].direction = [0,-1]

        self.semaphores = ap.AgentList(self,2, Semaphore)
        self.semaphores.step_time =  self.p.step_time
        self.semaphores.green_duration = self.p.green
        self.semaphores.yellow_duration = self.p.yellow
        self.semaphores.red_duration = self.p.red
        self.semaphores[0].direction = [0, 1]
        self.semaphores[1].direction = [0, -1]

        # Inicializa el entorno
        self.avenue = ap.Space(self, shape=[60, self.p.size], torus = True)
                
        # Agrega los semáforos al entorno
        self.avenue.add_agents(self.semaphores, random=True)
        self.avenue.move_to(self.semaphores[0], [10, self.p.size*0.5 + 5])
        self.avenue.move_to(self.semaphores[1], [50, self.p.size*0.5 - 5])

        # Agrega los autos al entorno
        self.avenue.add_agents(self.cars, random=True)
        for k in range(c_north):
            self.avenue.move_to(self.cars[k], [10, 10*(k+1)])
        
        for k in range(c_south):
            self.avenue.move_to(self.cars[k+c_north], [0, self.p.size - (k+1)*10])
            
        #CREACION DEL JSON
        
        
        self.data = {}
        self.data['cars'] = []
        self.data['semaphores'] = []
        self.data['frames'] = []
        
        
        with open('json_output', 'w') as file:
            for semaphore in self.semaphores:
                self.data['semaphores'].append({'id' : semaphore.id, 'dir' : semaphore.direction[1]})
            
            for car in self.cars:
                car_pos = self.avenue.positions[car]
                self.data['cars'].append({'id': car.id, 'dir': car.direction[1], 'x' : car_pos[0], 'z' :car_pos[1]})
        #cars, semaphores, frames 
        #inicializa ambos semaforos
        #inicializa coches
        
            json.dump(self.data, file, indent = 4)

    def step(self):
        """ Este método se invoca para actualizar el estado de la avenida. """        
        self.semaphores.update()
        
        self.cars.update_position()
        self.cars.update_speed()
        #cars, sempaphores list
        cars = []
        semaphores = []
        
        #SEMAPHORES
        semaphores.append({'id':self.semaphores[0].id, 'state': self.semaphores[0].state})
        semaphores.append({'id':self.semaphores[1].id, 'state': self.semaphores[1].state})
        
        #CAR
        #for every car append {id, pos, pos}
        
        for car in self.cars:
            position = self.avenue.positions[car]
            cars.append({'id':car.id, 'x': position[0], 'z': position[1], 'direction': car.direction[1]})
        
        self.data['frames'].append({'frame' : self.count, 'cars' : cars, 'semaphores' : semaphores, })
        
        with open('json_output', 'w') as file:
            json.dump(self.data, file, indent = 4)
            
        self.count += 1
        


## Funciones para visualización

In [132]:
def animation_plot_single(m, ax):    
    ax.set_title(f"Avenida t={m.t*m.p.step_time:.2f}")
    
    colors = ["green", "yellow", "red"]
    
    pos_s1 = m.avenue.positions[m.semaphores[0]]    
    ax.scatter(*pos_s1, s=20, c=colors[m.semaphores[0].state])
    
    pos_s2 = m.avenue.positions[m.semaphores[1]]    
    ax.scatter(*pos_s2, s=20, c=colors[m.semaphores[1].state])
    
    ax.set_xlim(0, m.avenue.shape[0])
    ax.set_ylim(0, m.avenue.shape[1])    
    
    for car in m.cars:
        pos_c = m.avenue.positions[car]    
        ax.scatter(*pos_c, s=20, c="black")
    
    ax.set_axis_off()
    ax.set_aspect('equal', 'box')
        
def animation_plot(m, p):    
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111)
    animation = ap.animate(m(p), fig, ax, animation_plot_single)
    return IPython.display.HTML(animation.to_jshtml(fps=20)) 

## Parámetros de la simulación

In [139]:
parameters = {
    'step_time': 0.1,    # Procentaje de área cubierta por árboles
    'size': 1000,        # Tamaño en metros de la avenida
    'green': 5,          # Duración de la luz verde
    'yellow': 0,         # Duración de la luz amarilla
    'red': 5,           # Duración de la luz roja
    'cars': 10,          # Número de autos en la simulación
    'steps': 1000        # Numero de pasos de la simulación
}

## Simulación de una corrida

In [140]:
model = AvenueModel(parameters)
results = model.run()

Completed: 1000 steps
Run time: 0:02:56.139300
Simulation finished


## Visualización de una corrida

In [8]:
animation_plot(AvenueModel, parameters)


CAR
{0: 0.0}

CAR
{0: 0.0}

CAR
{0: 0.0}

CAR
{0: 0.0}

CAR
{0: 0.0}

CAR
{0: 0.0}

CAR
{0: 0.0}

CAR
{0: 0.0}

CAR
{0: 0.0}

CAR
{0: 0.0}

CAR
{0: -0.0}

CAR
{0: -0.0}

CAR
{0: -0.0}

CAR
{0: -0.0}

CAR
{0: -0.0}

CAR
{0: -0.0}

CAR
{0: -0.0}

CAR
{0: -0.0}

CAR
{0: -0.0}

CAR
{0: -0.0}

 FRAME0

CAR
{0: 0.0, 1: 0.0}

CAR
{0: 0.0, 1: 0.0}

CAR
{0: 0.0, 1: 0.0}

CAR
{0: 0.0, 1: 0.0}

CAR
{0: 0.0, 1: 0.0}

CAR
{0: 0.0, 1: 0.0}

CAR
{0: 0.0, 1: 0.0}

CAR
{0: 0.0, 1: 0.0}

CAR
{0: 0.0, 1: 0.0}

CAR
{0: 0.0, 1: 0.5}

CAR
{0: -0.0, 1: -0.0}

CAR
{0: -0.0, 1: -0.0}

CAR
{0: -0.0, 1: -0.0}

CAR
{0: -0.0, 1: -0.0}

CAR
{0: -0.0, 1: -0.0}

CAR
{0: -0.0, 1: -0.0}

CAR
{0: -0.0, 1: -0.0}

CAR
{0: -0.0, 1: -0.0}

CAR
{0: -0.0, 1: -0.0}

CAR
{0: -0.0, 1: -0.5}

 FRAME1

CAR
{0: 0.0, 1: 0.0, 2: 0.0}

CAR
{0: 0.0, 1: 0.0, 2: 0.0}

CAR
{0: 0.0, 1: 0.0, 2: 0.0}

CAR
{0: 0.0, 1: 0.0, 2: 0.0}

CAR
{0: 0.0, 1: 0.0, 2: 0.0}

CAR
{0: 0.0, 1: 0.0, 2: 0.0}

CAR
{0: 0.0, 1: 0.0, 2: 0.0}

CAR
{0: 0.0, 1: 0.0, 


CAR
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0}

CAR
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0}

CAR
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0}

CAR
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0}

CAR
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0}

CAR
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0}

CAR
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0}

CAR
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0}

CAR
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0}

CAR
{0: 0.0, 1: 0.5, 2: 1.0, 3: 1.5, 4: 2.0, 5: 2.5, 6: 3.0, 7: 3.5, 8: 4.0, 9: 4.5}

CAR
{0: -0.0, 1: -0.0, 2: -0.0, 3: -0.0, 4: -0.0, 5: -0.0, 6: -0.0, 7: -0.0, 8: -0.0, 9: -0.0}

CAR
{0: -0.0, 1: -0.0, 2: -0.0, 3: -0.0, 4:


CAR
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0, 10: 0.0, 11: 0.0, 12: 0.0, 13: 0.0}

CAR
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0, 10: 0.0, 11: 0.0, 12: 0.0, 13: 0.0}

CAR
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0, 10: 0.0, 11: 0.0, 12: 0.0, 13: 0.0}

CAR
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0, 10: 0.0, 11: 0.0, 12: 0.0, 13: 0.0}

CAR
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0, 10: 0.0, 11: 0.0, 12: 0.0, 13: 0.0}

CAR
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0, 10: 0.0, 11: 0.0, 12: 0.0, 13: 0.0}

CAR
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0, 10: 0.0, 11: 0.0, 12: 0.0, 13: 0.0}

CAR
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0, 10: 0.0, 11: 0.0, 12: 0.0, 13: 0.0}

CAR
{0: 0.0, 1: 0.0, 2:


CAR
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0, 10: 0.0, 11: 0.0, 12: 0.0, 13: 0.0, 14: 0.0, 15: 0.0, 16: 0.0}

CAR
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0, 10: 0.0, 11: 0.0, 12: 0.0, 13: 0.0, 14: 0.0, 15: 0.0, 16: 0.0}

CAR
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0, 10: 0.0, 11: 0.0, 12: 0.0, 13: 0.0, 14: 0.0, 15: 0.0, 16: 0.0}

CAR
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0, 10: 0.0, 11: 0.0, 12: 0.0, 13: 0.0, 14: 0.0, 15: 0.0, 16: 0.0}

CAR
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0, 10: 0.0, 11: 0.0, 12: 0.0, 13: 0.0, 14: 0.0, 15: 0.0, 16: 0.0}

CAR
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0, 10: 0.0, 11: 0.0, 12: 0.0, 13: 0.0, 14: 0.0, 15: 0.0, 16: 0.0}

CAR
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0, 10: 0.0, 11: 0.0, 12


CAR
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0, 10: 0.0, 11: 0.0, 12: 0.0, 13: 0.0, 14: 0.0, 15: 0.0, 16: 0.0, 17: 0.0, 18: 0.0, 19: 0.0}

CAR
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0, 10: 0.0, 11: 0.0, 12: 0.0, 13: 0.0, 14: 0.0, 15: 0.0, 16: 0.0, 17: 0.0, 18: 0.0, 19: 0.0}

CAR
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0, 10: 0.0, 11: 0.0, 12: 0.0, 13: 0.0, 14: 0.0, 15: 0.0, 16: 0.0, 17: 0.0, 18: 0.0, 19: 0.0}

CAR
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0, 10: 0.0, 11: 0.0, 12: 0.0, 13: 0.0, 14: 0.0, 15: 0.0, 16: 0.0, 17: 0.0, 18: 0.0, 19: 0.0}

CAR
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0, 10: 0.0, 11: 0.0, 12: 0.0, 13: 0.0, 14: 0.0, 15: 0.0, 16: 0.0, 17: 0.0, 18: 0.0, 19: 0.0}

CAR
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0, 10: 0.0, 11: 0.0, 12: 0.0, 13: 0.0

...


CAR
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0, 10: 0.0, 11: 0.0, 12: 0.0, 13: 0.0, 14: 0.0, 15: 0.0, 16: 0.0, 17: 0.0, 18: 0.0, 19: 0.0, 20: 0.0, 21: 0.0, 22: 0.0, 23: 0.0, 24: 0.0, 25: 0.0, 26: 0.0, 27: 0.0, 28: 0.0, 29: 0.0, 30: 0.0, 31: 0.0, 32: 0.0, 33: 0.0, 34: 0.0, 35: 0.0, 36: 0.0, 37: 0.0, 38: 0.0, 39: 0.0, 40: 0.0, 41: 0.0, 42: 0.0}

CAR
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0, 10: 0.0, 11: 0.0, 12: 0.0, 13: 0.0, 14: 0.0, 15: 0.0, 16: 0.0, 17: 0.0, 18: 0.0, 19: 0.0, 20: 0.0, 21: 0.0, 22: 0.0, 23: 0.0, 24: 0.0, 25: 0.0, 26: 0.0, 27: 0.0, 28: 0.0, 29: 0.0, 30: 0.0, 31: 0.0, 32: 0.0, 33: 0.0, 34: 0.0, 35: 0.0, 36: 0.0, 37: 0.0, 38: 0.0, 39: 0.0, 40: 0.0, 41: 0.0, 42: 0.0}

CAR
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0, 10: 0.0, 11: 0.0, 12: 0.0, 13: 0.0, 14: 0.0, 15: 0.0, 16: 0.0, 17: 0.0, 18: 0.0, 19: 0.0, 20: 0.0, 21: 0.0, 22: 0.0, 23: 0.0, 24: 0.0, 25: 0.0, 26: 

CAR
{0: -0.0, 1: -0.5, 2: -1.0, 3: -1.5, 4: -2.0, 5: -2.5, 6: -3.0, 7: -3.5, 8: -4.0, 9: -4.5, 10: -5.0, 11: -5.5, 12: -6.0, 13: -6.5, 14: -7.0, 15: -7.5, 16: -8.0, 17: -8.5, 18: -9.0, 19: -9.5, 20: -10.0, 21: -10.5, 22: -11.0, 23: -11.5, 24: -12.0, 25: -12.5, 26: -13.0, 27: -13.5, 28: -14.0, 29: -14.5, 30: -15.0, 31: -15.5, 32: -16.0, 33: -16.5, 34: -17.0, 35: -17.5, 36: -18.0, 37: -18.5, 38: -16.5, 39: -17.0, 40: -17.5, 41: -18.0, 42: -18.5, 43: -19.0}

 FRAME43

CAR
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0, 10: 0.0, 11: 0.0, 12: 0.0, 13: 0.0, 14: 0.0, 15: 0.0, 16: 0.0, 17: 0.0, 18: 0.0, 19: 0.0, 20: 0.0, 21: 0.0, 22: 0.0, 23: 0.0, 24: 0.0, 25: 0.0, 26: 0.0, 27: 0.0, 28: 0.0, 29: 0.0, 30: 0.0, 31: 0.0, 32: 0.0, 33: 0.0, 34: 0.0, 35: 0.0, 36: 0.0, 37: 0.0, 38: 0.0, 39: 0.0, 40: 0.0, 41: 0.0, 42: 0.0, 43: 0.0, 44: 0.0}

CAR
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0, 10: 0.0, 11: 0.0, 12: 0.0, 13: 0.0, 14: 0.0,

CAR
{0: -0.0, 1: -0.5, 2: -1.0, 3: -1.5, 4: -2.0, 5: -2.5, 6: -3.0, 7: -3.5, 8: -4.0, 9: -4.5, 10: -5.0, 11: -5.5, 12: -6.0, 13: -6.5, 14: -7.0, 15: -7.5, 16: -8.0, 17: -8.5, 18: -9.0, 19: -9.5, 20: -10.0, 21: -10.5, 22: -11.0, 23: -11.5, 24: -12.0, 25: -12.5, 26: -13.0, 27: -13.5, 28: -14.0, 29: -14.5, 30: -15.0, 31: -15.5, 32: -16.0, 33: -16.5, 34: -17.0, 35: -17.5, 36: -18.0, 37: -18.5, 38: -16.5, 39: -17.0, 40: -17.5, 41: -18.0, 42: -18.5, 43: -19.0, 44: -19.5, 45: -20.0}

 FRAME45

CAR
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0, 10: 0.0, 11: 0.0, 12: 0.0, 13: 0.0, 14: 0.0, 15: 0.0, 16: 0.0, 17: 0.0, 18: 0.0, 19: 0.0, 20: 0.0, 21: 0.0, 22: 0.0, 23: 0.0, 24: 0.0, 25: 0.0, 26: 0.0, 27: 0.0, 28: 0.0, 29: 0.0, 30: 0.0, 31: 0.0, 32: 0.0, 33: 0.0, 34: 0.0, 35: 0.0, 36: 0.0, 37: 0.0, 38: 0.0, 39: 0.0, 40: 0.0, 41: 0.0, 42: 0.0, 43: 0.0, 44: 0.0, 45: 0.0, 46: 0.0}

CAR
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0, 10: 


CAR
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0, 10: 0.0, 11: 0.0, 12: 0.0, 13: 0.0, 14: 0.0, 15: 0.0, 16: 0.0, 17: 0.0, 18: 0.0, 19: 0.0, 20: 0.0, 21: 0.0, 22: 0.0, 23: 0.0, 24: 0.0, 25: 0.0, 26: 0.0, 27: 0.0, 28: 0.0, 29: 0.0, 30: 0.0, 31: 0.0, 32: 0.0, 33: 0.0, 34: 0.0, 35: 0.0, 36: 0.0, 37: 0.0, 38: 0.0, 39: 0.0, 40: 0.0, 41: 0.0, 42: 0.0, 43: 0.0, 44: 0.0, 45: 0.0, 46: 0.0, 47: 0.0}

CAR
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0, 10: 0.0, 11: 0.0, 12: 0.0, 13: 0.0, 14: 0.0, 15: 0.0, 16: 0.0, 17: 0.0, 18: 0.0, 19: 0.0, 20: 0.0, 21: 0.0, 22: 0.0, 23: 0.0, 24: 0.0, 25: 0.0, 26: 0.0, 27: 0.0, 28: 0.0, 29: 0.0, 30: 0.0, 31: 0.0, 32: 0.0, 33: 0.0, 34: 0.0, 35: 0.0, 36: 0.0, 37: 0.0, 38: 0.0, 39: 0.0, 40: 0.0, 41: 0.0, 42: 0.0, 43: 0.0, 44: 0.0, 45: 0.0, 46: 0.0, 47: 0.0}

CAR
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0, 10: 0.0, 11: 0.0, 12: 0.0, 13: 0.0, 14: 0.0, 15: 0.0, 16: 


CAR
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0, 10: 0.0, 11: 0.0, 12: 0.0, 13: 0.0, 14: 0.0, 15: 0.0, 16: 0.0, 17: 0.0, 18: 0.0, 19: 0.0, 20: 0.0, 21: 0.0, 22: 0.0, 23: 0.0, 24: 0.0, 25: 0.0, 26: 0.0, 27: 0.0, 28: 0.0, 29: 0.0, 30: 0.0, 31: 0.0, 32: 0.0, 33: 0.0, 34: 0.0, 35: 0.0, 36: 0.0, 37: 0.0, 38: 0.0, 39: 0.0, 40: 0.0, 41: 0.0, 42: 0.0, 43: 0.0, 44: 0.0, 45: 0.0, 46: 0.0, 47: 0.0, 48: 0.0}

CAR
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0, 10: 0.0, 11: 0.0, 12: 0.0, 13: 0.0, 14: 0.0, 15: 0.0, 16: 0.0, 17: 0.0, 18: 0.0, 19: 0.0, 20: 0.0, 21: 0.0, 22: 0.0, 23: 0.0, 24: 0.0, 25: 0.0, 26: 0.0, 27: 0.0, 28: 0.0, 29: 0.0, 30: 0.0, 31: 0.0, 32: 0.0, 33: 0.0, 34: 0.0, 35: 0.0, 36: 0.0, 37: 0.0, 38: 0.0, 39: 0.0, 40: 0.0, 41: 0.0, 42: 0.0, 43: 0.0, 44: 0.0, 45: 0.0, 46: 0.0, 47: 0.0, 48: 0.0}

CAR
{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0, 10: 0.0, 11: 0.0, 12: 0.0, 13: 0.0, 14: 